<a href="https://colab.research.google.com/github/Fiarma5233/Data-Science/blob/master/Projects_Sentiment_Analysis_for_Customer_Feedback_Product_and_Service_Improvements_with_Precision_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**OBJECTIVES**

Create a "Text to Vader" pipeline

Create a "Text to Stopwords remover to Vader" pipeline

Create a "Text to Stopwords remover to Bag of words to custom model"

Create a "Text to Stopwords remover to TF-IDF to customer model"

**WORKFLOW**

Data Collection And Loading

Text Processing

Model Creation

Model Evaluation And Selection

In [1]:
#Import libraries
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


DATA LOADING

In [2]:
!pip install gdown
# importing gdown
import gdown

#https://drive.google.com/file/d/1-63mzAYJJDpEKKgWSmIarte2HngnzY-n/view?usp=sharing
file1_id = "1-63mzAYJJDpEKKgWSmIarte2HngnzY-n" # using actual id
#output_file
output_file1 = "train_dataset.csv"
#download  the file
gdown.download(id=file1_id, output=output_file1, quiet=False)

train_dataset = pd.read_csv(output_file1)




#https://drive.google.com/file/d/1lHY0JIH_Vq3Tw1-jovccnqzFPgAhxEvg/view?usp=sharing
file2_id = "1lHY0JIH_Vq3Tw1-jovccnqzFPgAhxEvg" # using actual id
#output_file
output_file2 = "test_dataset.csv"
#download  the file
gdown.download(id=file2_id, output=output_file2, quiet=False)

test_dataset = pd.read_csv(output_file2)


Downloading...
From (original): https://drive.google.com/uc?id=1-63mzAYJJDpEKKgWSmIarte2HngnzY-n
From (redirected): https://drive.google.com/uc?id=1-63mzAYJJDpEKKgWSmIarte2HngnzY-n&confirm=t&uuid=3b9a8219-31ce-40e3-a53f-4dbbc23372f3
To: /content/train_dataset.csv
100%|██████████| 1.61G/1.61G [00:30<00:00, 53.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1lHY0JIH_Vq3Tw1-jovccnqzFPgAhxEvg
From (redirected): https://drive.google.com/uc?id=1lHY0JIH_Vq3Tw1-jovccnqzFPgAhxEvg&confirm=t&uuid=67205aeb-282c-41aa-8c01-1ebde26dba18
To: /content/test_dataset.csv
100%|██████████| 178M/178M [00:05<00:00, 33.3MB/s]


In [3]:
# check train dataset info
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600010 entries, 0 to 3600009
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   labels  object
 1   text    object
dtypes: object(2)
memory usage: 54.9+ MB


In [4]:
# check test dataset info

test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   labels  400000 non-null  object
 1   text    400000 non-null  object
dtypes: object(2)
memory usage: 6.1+ MB


In [5]:
# looking at the first 10 rows
train_dataset.head(10)

,labels,text
0,__label__2,Stuning even for the non-gamer: This sound tra...
1,__label__2,The best soundtrack ever to anything.: I'm rea...
2,__label__2,Amazing!: This soundtrack is my favorite music...
3,__label__2,Excellent Soundtrack: I truly like this soundt...
4,__label__2,"Remember, Pull Your Jaw Off The Floor After He..."
5,__label__2,an absolute masterpiece: I am quite sure any o...
6,__label__1,"Buyer beware: This is a self-published book, a..."
7,__label__2,Glorious story: I loved Whisper of the wicked ...
8,__label__2,A FIVE STAR BOOK: I just finished reading Whis...
9,__label__2,Whispers of the Wicked Saints: This was a easy...


In [6]:
# getting the row  at index 6
print(train_dataset.iloc[6]['text'])

Buyer beware: This is a self-published book, and if you want to know why--read a few paragraphs! Those 5 star reviews must have been written by Ms. Haddon's family and friends--or perhaps, by herself! I can't imagine anyone reading the whole thing--I spent an evening with the book and a friend and we were in hysterics reading bits and pieces of it to one another. It is most definitely bad enough to be entered into some kind of a "worst book" contest. I can't believe Amazon even sells this kind of thing. Maybe I can offer them my 8th grade term paper on "To Kill a Mockingbird"--a book I am quite sure Ms. Haddon never heard of. Anyway, unless you are in a mood to send a book to someone as a joke---stay far, far away from this one!


In [7]:
# list of unique classes in labels column
train_dataset['labels'].unique()

array(['__label__2', '__label__1'], dtype=object)

In [8]:
# in mapping ==> old value : new value

mapping = {
    '__label__1':'negative',
    '__label__2':'positive'

}

In [9]:
# mapping labels columns / changer les valeurs de la colonne "labels"
train_dataset['labels'] = train_dataset['labels'].map(mapping)

In [10]:
# repeating for the test dataset
test_dataset['labels'] = test_dataset['labels'].map(mapping)

In [11]:
train_dataset.head(10)

,labels,text
0,positive,Stuning even for the non-gamer: This sound tra...
1,positive,The best soundtrack ever to anything.: I'm rea...
2,positive,Amazing!: This soundtrack is my favorite music...
3,positive,Excellent Soundtrack: I truly like this soundt...
4,positive,"Remember, Pull Your Jaw Off The Floor After He..."
5,positive,an absolute masterpiece: I am quite sure any o...
6,negative,"Buyer beware: This is a self-published book, a..."
7,positive,Glorious story: I loved Whisper of the wicked ...
8,positive,A FIVE STAR BOOK: I just finished reading Whis...
9,positive,Whispers of the Wicked Saints: This was a easy...


**TEXT** **PROCESSING**

In [12]:
# remove stopwords
# bag  of words
#TF-IDF (Term Frequency - Inverse Document Frequency)

In [13]:
text = "this is an example with some  stopwords, i  love this product, it is good "

In [14]:
# tokenize the split : split into list of words
words = nltk.word_tokenize(text)

In [15]:
words

['this',
 'is',
 'an',
 'example',
 'with',
 'some',
 'stopwords',
 ',',
 'i',
 'love',
 'this',
 'product',
 ',',
 'it',
 'is',
 'good']

In [16]:
# filtering out stopwords
filter_words = [word for word in words if word not in stop_words]

In [17]:
filter_words

['example', 'stopwords', ',', 'love', 'product', ',', 'good']

In [18]:
# reconstruct the list to a sentence using join method
filtered_text = " ".join(filter_words)

In [19]:
filtered_text

'example stopwords , love product , good'

In [20]:

def remove_stopwords(text):

  """
    this function takes in a sentence
    tokenizes the sentence
    filters out the stopwords and returns a more compact sentence

  """
  words = nltk.word_tokenize(text)
  filter_words = [word for word in words if word not in stop_words]
  filtered_text = " ".join(filter_words)
  return filtered_text

In [21]:
remove_stopwords(text)

'example stopwords , love product , good'

In [22]:
train_dataset['text'].head(10).apply(remove_stopwords)

0    Stuning even non-gamer : This sound track beau...
1    The best soundtrack ever anything . : I 'm rea...
2    Amazing ! : This soundtrack favorite music tim...
3    Excellent Soundtrack : I truly like soundtrack...
4    Remember , Pull Your Jaw Off The Floor After H...
5    absolute masterpiece : I quite sure actually t...
6    Buyer beware : This self-published book , want...
7    Glorious story : I loved Whisper wicked saints...
8    A FIVE STAR BOOK : I finished reading Whisper ...
9    Whispers Wicked Saints : This easy read book m...
Name: text, dtype: object

In [23]:
from tqdm import tqdm

In [24]:
total_rows = len(train_dataset)
tqdm.pandas(total = total_rows)
train_dataset['stop words']=train_dataset['text'].progress_apply(remove_stopwords)

100%|██████████| 3600010/3600010 [1:05:51<00:00, 911.11it/s] 


In [25]:
total_rows = len(test_dataset)
tqdm.pandas(total = total_rows)
test_dataset['stop words']=test_dataset['text'].progress_apply(remove_stopwords)

100%|██████████| 400000/400000 [07:17<00:00, 914.03it/s]


In [26]:
train_dataset

,labels,text,stop words
0,positive,Stuning even for the non-gamer: This sound tra...,Stuning even non-gamer : This sound track beau...
1,positive,The best soundtrack ever to anything.: I'm rea...,The best soundtrack ever anything . : I 'm rea...
2,positive,Amazing!: This soundtrack is my favorite music...,Amazing ! : This soundtrack favorite music tim...
3,positive,Excellent Soundtrack: I truly like this soundt...,Excellent Soundtrack : I truly like soundtrack...
4,positive,"Remember, Pull Your Jaw Off The Floor After He...","Remember , Pull Your Jaw Off The Floor After H..."
...,...,...,...
3600005,negative,Don't do it!!: The high chair looks great when...,Do n't ! ! : The high chair looks great first ...
3600006,negative,"Looks nice, low functionality: I have used thi...","Looks nice , low functionality : I used highch..."
3600007,negative,"compact, but hard to clean: We have a small ho...","compact , hard clean : We small house , really..."
3600008,negative,what is it saying?: not sure what this book is...,saying ? : sure book supposed . It really reha...


A "**Bag of Words**" (BoW) is a simple and commonly used technique in **Natural Language Processing** (NLP) and text analysis to represent text data as numerical features. It is used to transform collection of text documents into a format  that can be pocessed by machine learning algotithms. The idea behind the Bag of Words model is to disregard the order and and structureof words in a text and focus only on the frequecy of each word's occurrence.

The key idea is that the order or words and the grammatical structure of sentence is ignored, and the analysis is purely based on the presence or absence of specific words and their frequencies.


**TF-IDF**, wihch srands for "**Term Frequency-Inverse Document Frequency**", is a numerical statistic used in information retrieval and Natural Language Processing to evaluate the importance of a word within a document relative to a collection of documents, typical a corpus.

The TF-IDF  score provides a measure of how important a term is within a specific document and across a collection of documents. Terms that appear frequently in a document but rarely in other documents receive higher TF-IDF  scores, making them indicative of the content of that document.

In [27]:
from sklearn.feature_extraction.text import CountVectorizer # for creating bag of words
from sklearn.feature_extraction.text import TfidfVectorizer # for creating tf-idf

In [28]:
# creating bag of words from stop words column

vectorizer = CountVectorizer()
train_bow = vectorizer.fit_transform(train_dataset['stop words'])
test_bow = vectorizer.transform(test_dataset['stop words'])

In [29]:
# creating a tfidf matrix from stopwords
tfidf_vectorizer = TfidfVectorizer()
train_tfidf = tfidf_vectorizer.fit_transform(train_dataset['stop words'])
test_tfidf = tfidf_vectorizer.transform(test_dataset['stop words'])

**MODELING**

In [30]:
# vader on normal sentences
# vader on sentences without stop words
# custom on train_tfidf
# custom on train_bow

In [31]:
# vader on normal sentences
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [32]:
# from vader import  sentiment analyser
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [33]:
 # VADER: pretrained model for analyzing sentiment of sentences

In [34]:
analyser = SentimentIntensityAnalyzer()

In [35]:
example_text = "i love the  orange  flavor, good product"

In [36]:
sentiment_scores = analyser.polarity_scores(example_text)

In [37]:
compound_score = sentiment_scores['compound']

In [38]:
compound_score

0.7964

In [39]:
def analyse_sentiment(sentence):
  """

    takes a sentence
    get the sentiments scores using analyser
    return positive if compound score is greater than 0
    return negative if compound score is less than 0, else return negative

  """
  sentiment_scores = analyser.polarity_scores(sentence)
  compound_score = sentiment_scores['compound']
  if compound_score >= 0:
    return 'positive'

  else:
    return 'negative'

In [40]:
analyse_sentiment(example_text)

'positive'

In [41]:
# vader on text
test_dataset["vader_on_text"] = test_dataset["text"].apply(analyse_sentiment)

In [42]:
# vader on stopwords
test_dataset["vader_on_text_without_stopwords"] = test_dataset["stop words"].apply(analyse_sentiment)

In [43]:
# trainning customer models on bag of words and tf-idf

In [44]:
train_tfidf

<3600010x980808 sparse matrix of type '<class 'numpy.float64'>'
	with 135880670 stored elements in Compressed Sparse Row format>

In [45]:
train_bow

<3600010x980808 sparse matrix of type '<class 'numpy.int64'>'
	with 135880670 stored elements in Compressed Sparse Row format>

In [46]:
# multinominal Naive Bayes
from sklearn.naive_bayes import MultinomialNB

In [47]:
# creating classifier for tfidf and bow

classifier_bow = MultinomialNB()
classifier_tfidf = MultinomialNB()

In [48]:
# fit these models
classifier_bow.fit(train_bow, train_dataset['labels'])
classifier_tfidf.fit(train_tfidf, train_dataset['labels'])

MultinomialNB()

In [49]:
train_dataset['labels']

0          positive
1          positive
2          positive
3          positive
4          positive
             ...   
3600005    negative
3600006    negative
3600007    negative
3600008    negative
3600009    positive
Name: labels, Length: 3600010, dtype: object

In [53]:
#making predictions with classifier
test_dataset['bow'] = classifier_bow.predict(test_bow)

In [52]:
test_dataset['tfidf'] = classifier_tfidf.predict(test_tfidf)

In [54]:
test_dataset

,labels,text,stop words,vader_on_text,vader_on_text_without_stopwords,bow,tfidf
0,positive,Great CD: My lovely Pat has one of the GREAT v...,Great CD : My lovely Pat one GREAT voices gene...,positive,positive,positive,positive
1,positive,One of the best game music soundtracks - for a...,One best game music soundtracks - game I n't r...,positive,positive,positive,positive
2,negative,Batteries died within a year ...: I bought thi...,Batteries died within year ... : I bought char...,positive,positive,negative,negative
3,positive,"works fine, but Maha Energy is better: Check o...","works fine , Maha Energy better : Check Maha E...",positive,positive,negative,negative
4,positive,Great for the non-audiophile: Reviewed quite a...,Great non-audiophile : Reviewed quite bit comb...,positive,positive,positive,positive
...,...,...,...,...,...,...,...
399995,negative,Unbelievable- In a Bad Way: We bought this Tho...,Unbelievable- In Bad Way : We bought Thomas so...,positive,positive,negative,negative
399996,negative,"Almost Great, Until it Broke...: My son reciev...","Almost Great , Until Broke ... : My son reciev...",negative,positive,negative,negative
399997,negative,Disappointed !!!: I bought this toy for my son...,Disappointed ! ! ! : I bought toy son loves ``...,positive,positive,negative,negative
399998,positive,Classic Jessica Mitford: This is a compilation...,Classic Jessica Mitford : This compilation wid...,positive,positive,positive,positive


**MODEL EVALUATION : USING ACCURACY SCORES AND CLASSIFICATION REPORT**

In [55]:
from sklearn.metrics import accuracy_score, classification_report

In [57]:
# what are we evaluting
test_dataset.head()

,labels,text,stop words,vader_on_text,vader_on_text_without_stopwords,bow,tfidf
0,positive,Great CD: My lovely Pat has one of the GREAT v...,Great CD : My lovely Pat one GREAT voices gene...,positive,positive,positive,positive
1,positive,One of the best game music soundtracks - for a...,One best game music soundtracks - game I n't r...,positive,positive,positive,positive
2,negative,Batteries died within a year ...: I bought thi...,Batteries died within year ... : I bought char...,positive,positive,negative,negative
3,positive,"works fine, but Maha Energy is better: Check o...","works fine , Maha Energy better : Check Maha E...",positive,positive,negative,negative
4,positive,Great for the non-audiophile: Reviewed quite a...,Great non-audiophile : Reviewed quite bit comb...,positive,positive,positive,positive


In [58]:


vader_text_accuracy_score = accuracy_score(test_dataset['labels'], test_dataset['vader_on_text'])

In [60]:
vader_text_accuracy_score * 100

71.151

In [68]:
print(classification_report(test_dataset['labels'], test_dataset['vader_on_text']))

              precision    recall  f1-score   support

    negative       0.88      0.49      0.63    200000
    positive       0.65      0.93      0.76    200000

    accuracy                           0.71    400000
   macro avg       0.76      0.71      0.70    400000
weighted avg       0.76      0.71      0.70    400000



In [61]:
vader_text_without_stopwords_accuracy_score = accuracy_score(test_dataset['labels'], test_dataset['vader_on_text_without_stopwords'])

In [62]:
vader_text_without_stopwords_accuracy_score * 100

68.6955

In [69]:
print(classification_report(test_dataset['labels'], test_dataset['vader_on_text_without_stopwords']))

              precision    recall  f1-score   support

    negative       0.88      0.43      0.58    200000
    positive       0.62      0.94      0.75    200000

    accuracy                           0.69    400000
   macro avg       0.75      0.69      0.67    400000
weighted avg       0.75      0.69      0.67    400000



In [63]:
bow_accuracy_score = accuracy_score(test_dataset['labels'], test_dataset['bow'])

In [64]:
bow_accuracy_score * 100

85.3445

In [70]:
print(classification_report(test_dataset['labels'], test_dataset['bow']))

              precision    recall  f1-score   support

    negative       0.85      0.86      0.85    200000
    positive       0.86      0.84      0.85    200000

    accuracy                           0.85    400000
   macro avg       0.85      0.85      0.85    400000
weighted avg       0.85      0.85      0.85    400000



In [65]:
tfidf_accuracy_score = accuracy_score(test_dataset['labels'], test_dataset['tfidf'])

In [66]:
tfidf_accuracy_score * 100

84.68525

In [71]:
print(classification_report(test_dataset['labels'], test_dataset['tfidf']))

              precision    recall  f1-score   support

    negative       0.84      0.86      0.85    200000
    positive       0.86      0.83      0.84    200000

    accuracy                           0.85    400000
   macro avg       0.85      0.85      0.85    400000
weighted avg       0.85      0.85      0.85    400000



** CONCLUSION: ** The bestperforming model/pipeline  is the text to stopwords to bow to multinomialNB

In [74]:
def inference(text):
  """

    remove stopwords
    convert remaing   text to bag of  words
    pass the bag of words to classifier_bow
    return  predicted

  """

  filtered_text = remove_stopwords(text)

  bow = vectorizer.transform([filtered_text])

  inference = classifier_bow.predict(bow)[0]

  return inference

In [75]:
exemple_text = ' i love the orange flavor, it is good '

In [76]:
inference(exemple_text)

'positive'

In [77]:
example2 = ' i dislike this animal, it is dangerous'

In [78]:
inference(example2)

'negative'